In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
import pandas as pd

file_name = '/content/drive/My Drive/新生訓練/新訓HW2/ratings_small.csv'
data = pd.read_csv(file_name, encoding='utf-8', engine='python')
data.head(2)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [3]:
print(type(data['userId'][0]), type(data['movieId'][0]), type(data['rating'][0]), type(data['timestamp'][0]))

<class 'numpy.int64'> <class 'numpy.int64'> <class 'numpy.float64'> <class 'numpy.int64'>


In [13]:
data.isna().sum()
# 無須處理空值

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [14]:
data['rating'] = data['rating'] / data['rating'].max()
print(data['rating'].max())
data.head(2)

1.0


,userId,movieId,rating,timestamp
0,1,31,0.5,1260759144
1,1,1029,0.6,1260759179


In [34]:
import numpy as np
from sklearn.model_selection import LeaveOneOut

target = data['rating']
del data['rating']

In [29]:
print(data.to_numpy())

[[         1         31 1260759144]
 [         1       1029 1260759179]
 [         1       1061 1260759182]
 ...
 [       671       6365 1070940363]
 [       671       6385 1070979663]
 [       671       6565 1074784724]]


In [52]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 46 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619427 sha256=d21b987894a6d4b6ebe6d05a12509f70b3538cb6cd657e793917f6089bee5437
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [58]:
from surprise import Reader
from surprise import Dataset
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

In [71]:
from surprise import SVD
algo = SVD(random_state=0)


In [78]:
from surprise.model_selection import LeaveOneOut
from collections import defaultdict
def GetTopN(predictions, n=10, minimumRating=4.0):
    topN = defaultdict(list)
    for userID, movieID, actualRating, estimatedRating, _ in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN
    
LOOCV = LeaveOneOut(n_splits=1, random_state=200)

for trainSet, testSet in LOOCV.split(data):
    # Train model without left-out ratings
    algo.fit(trainSet)
    # Predicts ratings for left-out ratings only
    leftOutPredictions = algo.test(testSet)
    # Build predictions for all ratings not in the training set
    bigTestSet = trainSet.build_anti_testset()
    allPredictions = algo.test(bigTestSet)
    # Compute top 10 recs for each user
    topNPredicted = GetTopN(allPredictions, n=10)

In [82]:
print(topNPredicted[20])

[(5971, 4.297507417881327), (926, 4.159679468752792), (1172, 4.14926923963846), (91529, 4.086330574759832), (741, 4.080218799236677), (1217, 4.073908900631712), (33166, 4.027912737831531), (1361, 4.014758567801407), (1254, 4.003840770278485)]


In [80]:
def HitRate(topNPredicted, leftOutPredictions):
    hits = 0
    total = 0

 # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top 10 for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit) :
            hits += 1

        total += 1

    # Compute overall precision
    return hits/total
print("\nHit Rate: ", HitRate(topNPredicted, leftOutPredictions))


Hit Rate:  0.020864381520119227
